**What I want to do:** Get suburb information from microburbs (fantastic site btw) and understand which place would be ideal for us to move next.

Microburbs is a fantastic site that has aggregated useful information from several government sources as well as made intelligent derivations (pretty much what I would have done with the raw information - only more detailed and smarter!). However, the only way to see information in microburbs is through a map hover feature which makes it really hard to see and compare information from  all the surrounding suburbs of my suburb of interest. So let me get information of all the suburbs and then simply compare them side by side.

## Admin stuff

In [63]:
import bs4
from bs4 import BeautifulSoup
from urllib2 import urlopen
import pandas as pd
import re
import numpy as np
from time import sleep
%matplotlib inline
import pylab as plt
import os
import warnings

## Source suburb html

I hovered over the inner west region of Sydney in microburbs.com.au and got the areas-list element upon inspection. Will go ahead and store this in a variable and extract info from the html.

In [153]:
baseid = 'https://www.microburbs.com.au'

In [3]:
innerwest_html_codedump = """
<p class="areas_list"><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132705" class="btn-link" style="display: inline;">1132705</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132709" class="btn-link" style="display: inline;">1132709</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132710" class="btn-link" style="display: inline;">1132710</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132711" class="btn-link" style="display: inline;">1132711</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132712" class="btn-link" style="display: inline;">1132712</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132713" class="btn-link" style="display: inline;">1132713</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132715" class="btn-link" style="display: inline;">1132715</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132716" class="btn-link" style="display: inline;">1132716</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132717" class="btn-link" style="display: inline;">1132717</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132718" class="btn-link" style="display: inline;">1132718</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132719" class="btn-link" style="display: inline;">1132719</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132720" class="btn-link" style="display: inline;">1132720</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132722" class="btn-link" style="display: inline;">1132722</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132723" class="btn-link" style="display: inline;">1132723</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132729" class="btn-link" style="display: inline;">1132729</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132730" class="btn-link" style="display: inline;">1132730</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132732" class="btn-link" style="display: inline;">1132732</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132733" class="btn-link" style="display: inline;">1132733</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132734" class="btn-link" style="display: inline;">1132734</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132735" class="btn-link" style="display: inline;">1132735</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132736" class="btn-link" style="display: inline;">1132736</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132750" class="btn-link" style="display: inline;">1132750</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlington-%28Sydney----NSW%29/1133201" class="btn-link" style="display: inline;">1133201</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133202" class="btn-link" style="display: inline;">1133202</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlington-%28Sydney----NSW%29/1133205" class="btn-link" style="display: inline;">1133205</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlington-%28Sydney----NSW%29/1133209" class="btn-link" style="display: inline;">1133209</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlington-%28Sydney----NSW%29/1133210" class="btn-link" style="display: inline;">1133210</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlington-%28Sydney----NSW%29/1133212" class="btn-link" style="display: inline;">1133212</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133213" class="btn-link" style="display: inline;">1133213</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133214" class="btn-link" style="display: inline;">1133214</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133215" class="btn-link" style="display: inline;">1133215</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133216" class="btn-link" style="display: inline;">1133216</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133217" class="btn-link" style="display: inline;">1133217</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133225" class="btn-link" style="display: inline;">1133225</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133226" class="btn-link" style="display: inline;">1133226</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133231" class="btn-link" style="display: inline;">1133231</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133232" class="btn-link" style="display: inline;">1133232</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133233" class="btn-link" style="display: inline;">1133233</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133235" class="btn-link" style="display: inline;">1133235</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133236" class="btn-link" style="display: inline;">1133236</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133237" class="btn-link" style="display: inline;">1133237</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133238" class="btn-link" style="display: inline;">1133238</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133239" class="btn-link" style="display: inline;">1133239</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133240" class="btn-link" style="display: inline;">1133240</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133241" class="btn-link" style="display: inline;">1133241</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133242" class="btn-link" style="display: inline;">1133242</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133249" class="btn-link" style="display: inline;">1133249</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133250" class="btn-link" style="display: inline;">1133250</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133251" class="btn-link" style="display: inline;">1133251</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133252" class="btn-link" style="display: inline;">1133252</a><a href="/NSW/Sydney/City/City-of-Sydney/Eveleigh/1133508" class="btn-link" style="display: inline;">1133508</a><a href="/NSW/Sydney/City/City-of-Sydney/Redfern/1133509" class="btn-link" style="display: inline;">1133509</a><a href="/NSW/Sydney/City/City-of-Sydney/Redfern/1133510" class="btn-link" style="display: inline;">1133510</a><a href="/NSW/Sydney/City/City-of-Sydney/Redfern/1133525" class="btn-link" style="display: inline;">1133525</a><a href="/NSW/Sydney/City/City-of-Sydney/Redfern/1133526" class="btn-link" style="display: inline;">1133526</a><a href="/NSW/Sydney/City/City-of-Sydney/Eveleigh/1133542" class="btn-link" style="display: inline;">1133542</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139410" class="btn-link" style="display: inline;">1139410</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139411" class="btn-link" style="display: inline;">1139411</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139424" class="btn-link" style="display: inline;">1139424</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139501" class="btn-link" style="display: inline;">1139501</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139516" class="btn-link" style="display: inline;">1139516</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132602" class="btn-link" style="display: inline;">1132602</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132614" class="btn-link" style="display: inline;">1132614</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132615" class="btn-link" style="display: inline;">1132615</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132616" class="btn-link" style="display: inline;">1132616</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132617" class="btn-link" style="display: inline;">1132617</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132618" class="btn-link" style="display: inline;">1132618</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132619" class="btn-link" style="display: inline;">1132619</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132620" class="btn-link" style="display: inline;">1132620</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132621" class="btn-link" style="display: inline;">1132621</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132622" class="btn-link" style="display: inline;">1132622</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132623" class="btn-link" style="display: inline;">1132623</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132624" class="btn-link" style="display: inline;">1132624</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132625" class="btn-link" style="display: inline;">1132625</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132626" class="btn-link" style="display: inline;">1132626</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132627" class="btn-link" style="display: inline;">1132627</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132628" class="btn-link" style="display: inline;">1132628</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132629" class="btn-link" style="display: inline;">1132629</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132630" class="btn-link" style="display: inline;">1132630</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132631" class="btn-link" style="display: inline;">1132631</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132638" class="btn-link" style="display: inline;">1132638</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132639" class="btn-link" style="display: inline;">1132639</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132640" class="btn-link" style="display: inline;">1132640</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132642" class="btn-link" style="display: inline;">1132642</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132643" class="btn-link" style="display: inline;">1132643</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132648" class="btn-link" style="display: inline;">1132648</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132649" class="btn-link" style="display: inline;">1132649</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132650" class="btn-link" style="display: inline;">1132650</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132651" class="btn-link" style="display: inline;">1132651</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132652" class="btn-link" style="display: inline;">1132652</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132653" class="btn-link" style="display: inline;">1132653</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132654" class="btn-link" style="display: inline;">1132654</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132655" class="btn-link" style="display: inline;">1132655</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132656" class="btn-link" style="display: inline;">1132656</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132657" class="btn-link" style="display: inline;">1132657</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132658" class="btn-link" style="display: inline;">1132658</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132659" class="btn-link" style="display: inline;">1132659</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132660" class="btn-link" style="display: inline;">1132660</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132661" class="btn-link" style="display: inline;">1132661</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132662" class="btn-link" style="display: inline;">1132662</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132701" class="btn-link" style="display: inline;">1132701</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132702" class="btn-link" style="display: inline;">1132702</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132703" class="btn-link" style="display: inline;">1132703</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132704" class="btn-link" style="display: inline;">1132704</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132706" class="btn-link" style="display: inline;">1132706</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132707" class="btn-link" style="display: inline;">1132707</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132708" class="btn-link" style="display: inline;">1132708</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132725" class="btn-link" style="display: inline;">1132725</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132726" class="btn-link" style="display: inline;">1132726</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132727" class="btn-link" style="display: inline;">1132727</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132728" class="btn-link" style="display: inline;">1132728</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132731" class="btn-link" style="display: inline;">1132731</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132737" class="btn-link" style="display: inline;">1132737</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132738" class="btn-link" style="display: inline;">1132738</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132739" class="btn-link" style="display: inline;">1132739</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132740" class="btn-link" style="display: inline;">1132740</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132741" class="btn-link" style="display: inline;">1132741</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132742" class="btn-link" style="display: inline;">1132742</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132743" class="btn-link" style="display: inline;">1132743</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132744" class="btn-link" style="display: inline;">1132744</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132745" class="btn-link" style="display: inline;">1132745</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132746" class="btn-link" style="display: inline;">1132746</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132747" class="btn-link" style="display: inline;">1132747</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132748" class="btn-link" style="display: inline;">1132748</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Enmore-%28Marrickville----NSW%29/1132749" class="btn-link" style="display: inline;">1132749</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132801" class="btn-link" style="display: inline;">1132801</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132815" class="btn-link" style="display: inline;">1132815</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132816" class="btn-link" style="display: inline;">1132816</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132819" class="btn-link" style="display: inline;">1132819</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132820" class="btn-link" style="display: inline;">1132820</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132821" class="btn-link" style="display: inline;">1132821</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132822" class="btn-link" style="display: inline;">1132822</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133001" class="btn-link" style="display: inline;">1133001</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133002" class="btn-link" style="display: inline;">1133002</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133003" class="btn-link" style="display: inline;">1133003</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133004" class="btn-link" style="display: inline;">1133004</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133005" class="btn-link" style="display: inline;">1133005</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133006" class="btn-link" style="display: inline;">1133006</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133007" class="btn-link" style="display: inline;">1133007</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133008" class="btn-link" style="display: inline;">1133008</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133009" class="btn-link" style="display: inline;">1133009</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133010" class="btn-link" style="display: inline;">1133010</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133011" class="btn-link" style="display: inline;">1133011</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133012" class="btn-link" style="display: inline;">1133012</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133013" class="btn-link" style="display: inline;">1133013</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133014" class="btn-link" style="display: inline;">1133014</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133015" class="btn-link" style="display: inline;">1133015</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133016" class="btn-link" style="display: inline;">1133016</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133017" class="btn-link" style="display: inline;">1133017</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133018" class="btn-link" style="display: inline;">1133018</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133020" class="btn-link" style="display: inline;">1133020</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133021" class="btn-link" style="display: inline;">1133021</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133022" class="btn-link" style="display: inline;">1133022</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133023" class="btn-link" style="display: inline;">1133023</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133025" class="btn-link" style="display: inline;">1133025</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133026" class="btn-link" style="display: inline;">1133026</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133027" class="btn-link" style="display: inline;">1133027</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133028" class="btn-link" style="display: inline;">1133028</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133029" class="btn-link" style="display: inline;">1133029</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133030" class="btn-link" style="display: inline;">1133030</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133031" class="btn-link" style="display: inline;">1133031</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133032" class="btn-link" style="display: inline;">1133032</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133206" class="btn-link" style="display: inline;">1133206</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133207" class="btn-link" style="display: inline;">1133207</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133208" class="btn-link" style="display: inline;">1133208</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133218" class="btn-link" style="display: inline;">1133218</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133219" class="btn-link" style="display: inline;">1133219</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133220" class="btn-link" style="display: inline;">1133220</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133221" class="btn-link" style="display: inline;">1133221</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133222" class="btn-link" style="display: inline;">1133222</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133223" class="btn-link" style="display: inline;">1133223</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133224" class="btn-link" style="display: inline;">1133224</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133227" class="btn-link" style="display: inline;">1133227</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133228" class="btn-link" style="display: inline;">1133228</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133229" class="btn-link" style="display: inline;">1133229</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133230" class="btn-link" style="display: inline;">1133230</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133234" class="btn-link" style="display: inline;">1133234</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133243" class="btn-link" style="display: inline;">1133243</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133244" class="btn-link" style="display: inline;">1133244</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133245" class="btn-link" style="display: inline;">1133245</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133246" class="btn-link" style="display: inline;">1133246</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133247" class="btn-link" style="display: inline;">1133247</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133248" class="btn-link" style="display: inline;">1133248</a><a href="/NSW/Sydney/City/City-of-Sydney/Beaconsfield-%28NSW%29/1133816" class="btn-link" style="display: inline;">1133816</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139517" class="btn-link" style="display: inline;">1139517</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139408" class="btn-link" style="display: inline;">1139408</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139420" class="btn-link" style="display: inline;">1139420</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139421" class="btn-link" style="display: inline;">1139421</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139422" class="btn-link" style="display: inline;">1139422</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Lewisham-%28NSW%29/1139423" class="btn-link" style="display: inline;">1139423</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139425" class="btn-link" style="display: inline;">1139425</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139427" class="btn-link" style="display: inline;">1139427</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139428" class="btn-link" style="display: inline;">1139428</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139429" class="btn-link" style="display: inline;">1139429</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139430" class="btn-link" style="display: inline;">1139430</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139433" class="btn-link" style="display: inline;">1139433</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139434" class="btn-link" style="display: inline;">1139434</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139435" class="btn-link" style="display: inline;">1139435</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139436" class="btn-link" style="display: inline;">1139436</a><a href="/NSW/Sydney/Mascot/1132204" class="btn-link" style="display: inline;">1132204</a><a href="/NSW/Sydney/Mascot/1132216" class="btn-link" style="display: inline;">1132216</a><a href="/NSW/Sydney/Mascot/1132217" class="btn-link" style="display: inline;">1132217</a><a href="/NSW/Sydney/Mascot/1132219" class="btn-link" style="display: inline;">1132219</a><a href="/NSW/Sydney/Rosebery-%28NSW%29/1132242" class="btn-link" style="display: inline;">1132242</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132603" class="btn-link" style="display: inline;">1132603</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132604" class="btn-link" style="display: inline;">1132604</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132605" class="btn-link" style="display: inline;">1132605</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132606" class="btn-link" style="display: inline;">1132606</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132607" class="btn-link" style="display: inline;">1132607</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132608" class="btn-link" style="display: inline;">1132608</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132609" class="btn-link" style="display: inline;">1132609</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132612" class="btn-link" style="display: inline;">1132612</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132613" class="btn-link" style="display: inline;">1132613</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132632" class="btn-link" style="display: inline;">1132632</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132633" class="btn-link" style="display: inline;">1132633</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132634" class="btn-link" style="display: inline;">1132634</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132635" class="btn-link" style="display: inline;">1132635</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132636" class="btn-link" style="display: inline;">1132636</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132637" class="btn-link" style="display: inline;">1132637</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132641" class="btn-link" style="display: inline;">1132641</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132644" class="btn-link" style="display: inline;">1132644</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132645" class="btn-link" style="display: inline;">1132645</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132646" class="btn-link" style="display: inline;">1132646</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132647" class="btn-link" style="display: inline;">1132647</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132802" class="btn-link" style="display: inline;">1132802</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Sydenham-%28NSW%29/1132803" class="btn-link" style="display: inline;">1132803</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Sydenham-%28NSW%29/1132804" class="btn-link" style="display: inline;">1132804</a><a href="/NSW/Sydney/St-Peters-%28NSW%29/1132805" class="btn-link" style="display: inline;">1132805</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Sydenham-%28NSW%29/1132806" class="btn-link" style="display: inline;">1132806</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132807" class="btn-link" style="display: inline;">1132807</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132808" class="btn-link" style="display: inline;">1132808</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132809" class="btn-link" style="display: inline;">1132809</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132810" class="btn-link" style="display: inline;">1132810</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132813" class="btn-link" style="display: inline;">1132813</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132817" class="btn-link" style="display: inline;">1132817</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Sydenham-%28NSW%29/1132818" class="btn-link" style="display: inline;">1132818</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133019" class="btn-link" style="display: inline;">1133019</a><a href="/NSW/Sydney/Rosebery-%28NSW%29/1133819" class="btn-link" style="display: inline;">1133819</a><a href="/NSW/Sydney/Rosebery-%28NSW%29/1133827" class="btn-link" style="display: inline;">1133827</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136410" class="btn-link" style="display: inline;">1136410</a><a href="/NSW/Sydney/Mascot/1132203" class="btn-link" style="display: inline;">1132203</a><a href="/NSW/Sydney/Rosebery-%28NSW%29/1132241" class="btn-link" style="display: inline;">1132241</a><a href="/NSW/Sydney/Rosebery-%28NSW%29/1132244" class="btn-link" style="display: inline;">1132244</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132811" class="btn-link" style="display: inline;">1132811</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Tempe/1132812" class="btn-link" style="display: inline;">1132812</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136452" class="btn-link" style="display: inline;">1136452</a><a href="/NSW/Sydney/Inner-West/Leichhardt-Municipality/Leichhardt-%28NSW%29/1138807" class="btn-link" style="display: inline;">1138807</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139502" class="btn-link" style="display: inline;">1139502</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132601" class="btn-link" style="display: inline;">1132601</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139432" class="btn-link" style="display: inline;">1139432</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139426" class="btn-link" style="display: inline;">1139426</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132611" class="btn-link" style="display: inline;">1132611</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139409" class="btn-link" style="display: inline;">1139409</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139412" class="btn-link" style="display: inline;">1139412</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139437" class="btn-link" style="display: inline;">1139437</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136405" class="btn-link" style="display: inline;">1136405</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136424" class="btn-link" style="display: inline;">1136424</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136451" class="btn-link" style="display: inline;">1136451</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132721" class="btn-link" style="display: inline;">1132721</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlington-%28Sydney----NSW%29/1133211" class="btn-link" style="display: inline;">1133211</a><a href="/NSW/Sydney/Inner-West/Leichhardt-Municipality/Leichhardt-%28NSW%29/1138806" class="btn-link" style="display: inline;">1138806</a><a href="/NSW/Sydney/Inner-West/Leichhardt-Municipality/Leichhardt-%28NSW%29/1138809" class="btn-link" style="display: inline;">1138809</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Haberfield/1139524" class="btn-link" style="display: inline;">1139524</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132724" class="btn-link" style="display: inline;">1132724</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132714" class="btn-link" style="display: inline;">1132714</a><a href="/NSW/Sydney/Camperdown-%28NSW%29/1133203" class="btn-link" style="display: inline;">1133203</a><a href="/NSW/Sydney/City/City-of-Sydney/Chippendale/1133516" class="btn-link" style="display: inline;">1133516</a><a href="/NSW/Sydney/City/City-of-Sydney/Chippendale/1133528" class="btn-link" style="display: inline;">1133528</a><a href="/NSW/Sydney/Inner-West/Leichhardt-Municipality/Leichhardt-%28NSW%29/1138810" class="btn-link" style="display: inline;">1138810</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139509" class="btn-link" style="display: inline;">1139509</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139510" class="btn-link" style="display: inline;">1139510</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Marrickville/1132610" class="btn-link" style="display: inline;">1132610</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139404" class="btn-link" style="display: inline;">1139404</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139407" class="btn-link" style="display: inline;">1139407</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139431" class="btn-link" style="display: inline;">1139431</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139439" class="btn-link" style="display: inline;">1139439</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136403" class="btn-link" style="display: inline;">1136403</a><a href="/NSW/Sydney-Western-suburbs/City-of-Canterbury/Earlwood/1136449" class="btn-link" style="display: inline;">1136449</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139512" class="btn-link" style="display: inline;">1139512</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139514" class="btn-link" style="display: inline;">1139514</a><a href="/NSW/Sydney/Inner-West/Ashfield-Municipality/Summer-Hill-%28Ashfield----NSW%29/1139523" class="btn-link" style="display: inline;">1139523</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139413" class="btn-link" style="display: inline;">1139413</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Dulwich-Hill/1139438" class="btn-link" style="display: inline;">1139438</a></p>
"""

## Get information from the html

In [4]:
soup = BeautifulSoup(innerwest_html_codedump, "lxml")

In [5]:
# prototype the logic
counter = 0
for a in soup.find_all('a', href=True):
    counter+=1
    print a.string, a['href']
    if counter == 5:
        break

1132705 /NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132705
1132709 /NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132709
1132710 /NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132710
1132711 /NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132711
1132712 /NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132712


## Get the information into a list and push to df

In [6]:
burbid=[]; burblink = []
for a in soup.find_all('a', href=True):
    burbid.append(a.string.string.encode('utf-8'))
    burblink.append(a['href'])

In [7]:
burbs = pd.DataFrame({'burbid':burbid,
                      'burblink':burblink})
burbs.head()

,burbid,burblink
0,1132705,/NSW/Sydney/Inner-West/Marrickville-Municipali...
1,1132709,/NSW/Sydney/Inner-West/Marrickville-Municipali...
2,1132710,/NSW/Sydney/Inner-West/Marrickville-Municipali...
3,1132711,/NSW/Sydney/Inner-West/Marrickville-Municipali...
4,1132712,/NSW/Sydney/Inner-West/Marrickville-Municipali...


In [154]:
location = 'Central'
df_blah = pd.DataFrame({'location':[location for x in burbid],'burbid':burbid,'burblink':burblink})
df_blah['burblink']=df_blah['burblink'].apply(lambda x: baseid+x)
df_blah.head()

,burbid,burblink,location
0,1132705,https://www.microburbs.com.au/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132705,Central
1,1132709,https://www.microburbs.com.au/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132709,Central
2,1132710,https://www.microburbs.com.au/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29...,Central
3,1132711,https://www.microburbs.com.au/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29...,Central
4,1132712,https://www.microburbs.com.au/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29...,Central


In [156]:
# write a function for this:
def get_burbid_and_burblink(location, html_codedump, baseid = 'https://www.microburbs.com.au'):
    soup = BeautifulSoup(html_codedump, "lxml")
    burbid=[]; burblink = []
    for a in soup.find_all('a', href=True):
        burbid.append(a.string.string.encode('utf-8'))
        burblink.append(a['href'])
    df_temp = pd.DataFrame({'location':[location for x in burbid],'burbid':burbid,'burblink':burblink})
    df_temp['burblink'] = df_temp['burblink'].apply(lambda x: baseid+x)
    return df_temp

## Explore individual burb

In [58]:
burbs[burbs.burbid=='1133526']

,burbid,burblink
54,1133526,/NSW/Sydney/City/City-of-Sydney/Redfern/1133526


In [59]:
pd.options.display.max_colwidth = 100
burbs.loc[burbs.burbid=='1133526', 'burblink'].tolist()

['/NSW/Sydney/City/City-of-Sydney/Redfern/1133526']

In [61]:
burb_full_link = ''.join(baseid+burbs.loc[burbs.burbid=='1133526', 'burblink'].tolist())

## Explore element inside code

In [127]:
html = urlopen(burb_full_link).read()  
soup = BeautifulSoup(html, 'lxml')

In [128]:
soup.findAll('div', attrs = {'class' : 'collapse', 'id': 'localprices-score-collapse'})

[]

In [130]:
soup.find('div',{'class':'col-sm-6 col-lg-5'}).findAll('div',{'id':"localprices-score-collapse"})

[]

Unfortunately, it looks like the website is coded with dynamic javascript getting generated during run time. Its going to be more tricky to scrape the data. So let me not do that. I'll simply get the html for microburbs manually, use the code above to get the burbids, and then run them one after the other.

## Collect micro suburb html

This is based on a visual inspection of which micro suburbs I'm interested in. I zoom into the microburbs.com.au map to whatever level of detail I want, and then I use the code below to grab a list of the burbids and thr burblinks for future inspection.

In [179]:
dict_burb_html = {
"Central_near_gym":"""
<p class="areas_list"><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133623" class="btn-link" style="display: inline;">1133623</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133627" class="btn-link" style="display: inline;">1133627</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133603" class="btn-link" style="display: inline;">1133603</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133610" class="btn-link" style="display: inline;">1133610</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133612" class="btn-link" style="display: inline;">1133612</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133613" class="btn-link" style="display: inline;">1133613</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133614" class="btn-link" style="display: inline;">1133614</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133615" class="btn-link" style="display: inline;">1133615</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133616" class="btn-link" style="display: inline;">1133616</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133624" class="btn-link" style="display: inline;">1133624</a><a href="/NSW/Sydney/City/City-of-Sydney/Haymarket/1133714" class="btn-link" style="display: inline;">1133714</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133617" class="btn-link" style="display: inline;">1133617</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133618" class="btn-link" style="display: inline;">1133618</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133622" class="btn-link" style="display: inline;">1133622</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlinghurst/1132904" class="btn-link" style="display: inline;">1132904</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlinghurst/1132908" class="btn-link" style="display: inline;">1132908</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133625" class="btn-link" style="display: inline;">1133625</a><a href="/NSW/Sydney/City/City-of-Sydney/Darlinghurst/1132905" class="btn-link" style="display: inline;">1132905</a><a href="/NSW/Sydney/City/City-of-Sydney/Surry-Hills/1133611" class="btn-link" style="display: inline;">1133611</a></p>
""",
"Stanmore_near_station":"""
<p class="areas_list"><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132709" class="btn-link" style="display: inline;">1132709</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132715" class="btn-link" style="display: inline;">1132715</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132729" class="btn-link" style="display: inline;">1132729</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132730" class="btn-link" style="display: inline;">1132730</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132735" class="btn-link" style="display: inline;">1132735</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132736" class="btn-link" style="display: inline;">1132736</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132737" class="btn-link" style="display: inline;">1132737</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132739" class="btn-link" style="display: inline;">1132739</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132740" class="btn-link" style="display: inline;">1132740</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132750" class="btn-link" style="display: inline;">1132750</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Petersham/1132708" class="btn-link" style="display: inline;">1132708</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132725" class="btn-link" style="display: inline;">1132725</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132738" class="btn-link" style="display: inline;">1132738</a><a href="/NSW/Sydney/Inner-West/Marrickville-Municipality/Stanmore-%28NSW%29/1132744" class="btn-link" style="display: inline;">1132744</a></p>
""",
"Erko_near_station":"""
<p class="areas_list"><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133002" class="btn-link" style="display: inline;">1133002</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133003" class="btn-link" style="display: inline;">1133003</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133004" class="btn-link" style="display: inline;">1133004</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133005" class="btn-link" style="display: inline;">1133005</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133006" class="btn-link" style="display: inline;">1133006</a><a href="/NSW/Sydney/City/City-of-Sydney/Alexandria/1133007" class="btn-link" style="display: inline;">1133007</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133025" class="btn-link" style="display: inline;">1133025</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133026" class="btn-link" style="display: inline;">1133026</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133027" class="btn-link" style="display: inline;">1133027</a><a href="/NSW/Sydney/City/City-of-Sydney/Erskineville/1133028" class="btn-link" style="display: inline;">1133028</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133223" class="btn-link" style="display: inline;">1133223</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133230" class="btn-link" style="display: inline;">1133230</a><a href="/NSW/Sydney/Newtown-%28NSW%29/1133243" class="btn-link" style="display: inline;">1133243</a><a href="/NSW/Sydney/City/City-of-Sydney/Eveleigh/1133542" class="btn-link" style="display: inline;">1133542</a></p>
"""
}

In [180]:
for dict_burb_html_key, dict_burb_html_item in dict_burb_html.items():
    print dict_burb_html_key

Stanmore_near_station
Erko_near_station
Central_near_gym


In [177]:
df_burbs = pd.DataFrame()
for dict_burb_html_key, dict_burb_html_item in dict_burb_html.items():
    df_burbs = pd.concat((df_burbs, get_burbid_and_burblink(dict_burb_html_key,dict_burb_html_item)))

In [178]:
df_burbs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 18
Data columns (total 3 columns):
burbid      47 non-null object
burblink    47 non-null object
location    47 non-null object
dtypes: object(3)
memory usage: 1.5+ KB


In [182]:
df_burbs.to_csv('microburb_info.csv')